In [2]:
import pandas as pd
import numpy as np
import folium
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import json


def func(x):
    d = {'count': x['건수'].sum()}
    return pd.Series(d, index=['count'])


def func2(x):
    d = {'diff': x['count'].diff()}
    return pd.Series(d, index=['diff'])


dataFrame = pd.read_csv("./yearInter.csv")

print(dataFrame)
print('----------------------------------------------')

      Unnamed: 0   구분  죄종 발생검거      건수        date
0              0   중부  살인   발생     1.0  2000-01-01
1              1   중부  살인   검거     1.0  2000-01-01
2              2   중부  강도   발생    17.0  2000-01-01
3              3   중부  강도   검거    15.0  2000-01-01
4              4   중부  강간   발생    14.0  2000-01-01
5              5   중부  강간   검거    14.0  2000-01-01
6              6   중부  절도   발생   601.0  2000-01-01
7              7   중부  절도   검거   569.0  2000-01-01
8              8   중부  폭력   발생  1783.0  2000-01-01
9              9   중부  폭력   검거  1762.0  2000-01-01
10            10   종로  살인   발생     4.0  2000-01-01
11            11   종로  살인   검거     4.0  2000-01-01
12            12   종로  강도   발생    24.0  2000-01-01
13            13   종로  강도   검거    36.0  2000-01-01
14            14   종로  강간   발생    18.0  2000-01-01
15            15   종로  강간   검거    18.0  2000-01-01
16            16   종로  절도   발생   644.0  2000-01-01
17            17   종로  절도   검거   385.0  2000-01-01
18            18   종로  폭력   발생 

In [3]:
# eachTopRank = dataFrame.describe(include=[np.object])
#
# print(eachTopRank)
# print('----------------------------------------------')
# 각 죄종별 범죄 발생/검거에 대한 상관관계를 구하고 싶다.
# 죄종별 발생/검거별 그룹화 필요
groupedDF = dataFrame.groupby(['발생검거', '죄종']).apply(func)
print(groupedDF)
print('----------------------------------------------')
resetGroupedDF = groupedDF.reset_index()
print(resetGroupedDF)
print('----------------------------------------------')
resetGroupedDF.to_csv('../resetGroupdDF.csv')

resetGroupedDF2 = pd.read_csv("../resetGroupdDF.csv")
groupedResetGroupedDF = resetGroupedDF2.groupby(['죄종'], sort=False).apply(func2)
print(groupedResetGroupedDF.reset_index())
print('----------------------------------------------')

             count
발생검거 죄종           
검거   강간    60749.0
     강도    17949.0
     살인     3611.0
     절도   442070.0
     폭력  1250621.0
발생   강간    78396.0
     강도    20292.0
     살인     3683.0
     절도   871779.0
     폭력  1394377.0
----------------------------------------------
  발생검거  죄종      count
0   검거  강간    60749.0
1   검거  강도    17949.0
2   검거  살인     3611.0
3   검거  절도   442070.0
4   검거  폭력  1250621.0
5   발생  강간    78396.0
6   발생  강도    20292.0
7   발생  살인     3683.0
8   발생  절도   871779.0
9   발생  폭력  1394377.0
----------------------------------------------
   죄종                                               diff
0  강간  4        NaN
9    17647.0
Name: count, dtype: ...
1  강도  4       NaN
9    2343.0
Name: count, dtype: fl...
2  살인    4     NaN
9    72.0
Name: count, dtype: float64
3  절도  4         NaN
9    429709.0
Name: count, dtype...
4  폭력  4         NaN
9    143756.0
Name: count, dtype...
----------------------------------------------


In [4]:
# print(dataFrame.corr())
# print('----------------------------------------------')
#
# print(dataFrame.cov())
# print('----------------------------------------------')

# 죄종/발생검거 컬럼 합치기
dataFrame['죄종/발생검거'] = dataFrame['죄종'].astype(str) + ' ' + dataFrame['발생검거']
print(dataFrame)
print('----------------------------------------------')

# 지역별 죄종/발생검거 현황 컬럼으로 pivot 시키기 (날짜 컬럼은 사라지므로 시계열로서는 의미가 없어지며 지역의 특성에 집중됨)
# 시계열 데이터로 분석하기에는 애초에 2000 ~ 2018년까지 19개 뿐이므로 좋은 분석 결과를 얻기 힘들 것으로 사료됨
# pivot 함수 적용 시, index 중복 오류가 뜸. pd.pivot_table로 대체 가능
# pivotDataFrame = dataFrame.pivot(index='구분', columns='죄종/발생검거', values='건수')
pivotDataFrame = pd.pivot_table(dataFrame, index='구분', columns='죄종/발생검거', values='건수', aggfunc=np.sum)
print(pivotDataFrame.reset_index())
print('----------------------------------------------')
pivotDataFrame.reset_index().to_csv('../pivotDataFrame.csv', encoding='utf-8')

      Unnamed: 0   구분  죄종 발생검거      건수        date 죄종/발생검거
0              0   중부  살인   발생     1.0  2000-01-01   살인 발생
1              1   중부  살인   검거     1.0  2000-01-01   살인 검거
2              2   중부  강도   발생    17.0  2000-01-01   강도 발생
3              3   중부  강도   검거    15.0  2000-01-01   강도 검거
4              4   중부  강간   발생    14.0  2000-01-01   강간 발생
5              5   중부  강간   검거    14.0  2000-01-01   강간 검거
6              6   중부  절도   발생   601.0  2000-01-01   절도 발생
7              7   중부  절도   검거   569.0  2000-01-01   절도 검거
8              8   중부  폭력   발생  1783.0  2000-01-01   폭력 발생
9              9   중부  폭력   검거  1762.0  2000-01-01   폭력 검거
10            10   종로  살인   발생     4.0  2000-01-01   살인 발생
11            11   종로  살인   검거     4.0  2000-01-01   살인 검거
12            12   종로  강도   발생    24.0  2000-01-01   강도 발생
13            13   종로  강도   검거    36.0  2000-01-01   강도 검거
14            14   종로  강간   발생    18.0  2000-01-01   강간 발생
15            15   종로  강간   검거    18.0  2000-01-01   강간 

In [13]:
# 아무곳이나 좌표 찍어서 지도로 보여주기
# map_osm = folium.Map(location=[45.5236, -122.6750], zoom_start=12)
# map_osm.save('test.html')

# 서울 구역 json 데이터로 표시해주기
seoul_geo_path = './pythonCode/seoul_area.json'
geo_str = json.load(open(seoul_geo_path, encoding='utf-8'))

pivotData = pd.read_csv('./pivotDataFrame.csv')
pivotData['구분'] = pivotData['구분'] + '구'
mapA = folium.Map(location=[37.5502, 126.982],
                  zoom_start=11,
                  tiles='Stamen Toner')
pivotData.set_index('구분')
print(pivotData)
print('----------------------------------------------')
print(pivotData.index)
mapA.choropleth(
                    geo_data=geo_str,
                    data=pivotData['강간 검거'],
                    columns=[pivotData['구분'], pivotData['강간 검거']],
                    fill_color='YlGnBu',
                    key_on='feature.id')

    Unnamed: 0    구분   강간 검거   강간 발생   강도 검거   강도 발생  살인 검거  살인 발생    절도 검거  \
0            0   강남구  3298.0  4400.0  1079.0  1418.0  134.0  139.0  17550.0   
1            1   강동구  2112.0  2386.0   849.0   828.0  129.0  130.0  18405.0   
2            2   강북구  2114.0  2522.0   796.0   779.0  135.0  142.0  13624.0   
3            3   강서구  2797.0  3300.0   921.0  1014.0  197.0  201.0  18146.0   
4            4   관악구  3241.0  4942.0   704.0   912.0  141.0  152.0  15971.0   
5            5   광진구  2988.0  3569.0   910.0  1006.0  162.0  150.0  22371.0   
6            6   구로구  2350.0  3856.0   641.0   751.0  201.0  202.0  15757.0   
7            7   금천구  1961.0  2244.0   676.0   865.0  171.0  168.0  15652.0   
8            8  남대문구   802.0   975.0   220.0   226.0   37.0   33.0   8896.0   
9            9   노원구  2034.0  2392.0   377.0   386.0  132.0  130.0  15397.0   
10          10   도봉구  1557.0  1645.0   380.0   395.0   90.0   90.0  11700.0   
11          11  동대문구  2254.0  2550.0   805.0   869.0

D:\1Research\Anaconda\envs\tensorflow_gpu\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [14]:
mapA